# Importing the libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [8]:
#Import the geographical coordinates of each postal code
df_paris_district = pd.read_csv("/Users/J-Luc/Downloads/Data science IBM/Cours 9 - Applied Data Science Capstone/ParisArrondissementsCoordonees.csv")
df_paris_district.head(5)

,Arrondissement,Latitude,Longitude
0,1,48.862497,2.336686
1,2,48.866451,2.341356
2,3,48.863642,2.361263
3,4,48.854662,2.362913
4,5,48.845428,2.344691


In [9]:
print(df_paris_district.shape)

(20, 3)


#### Use geopy library to get the latitude and longitude values of New York City

In [11]:
address = 'Paris, FR'

geolocator = Nominatim(user_agent="paris_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


#### Create a map of Paris with the 20 districts (center) superimposed on top

In [19]:
# create map of Paris using latitude and longitude values
map_paris = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, arr in zip(df_paris_district['Latitude'], df_paris_district['Longitude'], df_paris_district['Arrondissement']):
    label = '{}, Arrondissement {}'.format(address, arr)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

#### Define Foursquare Credentials and Version

In [20]:
CLIENT_ID = 'EMC0WNWPGRILVV5GXBNFYYWLH0OJIRSC5BJRVBBMPOHWADEU' # your Foursquare ID
CLIENT_SECRET = 'I31543KR20OXAXEUJTI5WEHN0F3RTH5LXMXL1IHYSXVETIHO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EMC0WNWPGRILVV5GXBNFYYWLH0OJIRSC5BJRVBBMPOHWADEU
CLIENT_SECRET:I31543KR20OXAXEUJTI5WEHN0F3RTH5LXMXL1IHYSXVETIHO


In [103]:
arr_latitude = df_paris_district.loc[0, 'Latitude'] # neighborhood latitude value
arr_longitude = df_paris_district.loc[0, 'Longitude'] # neighborhood longitude value

arr_name = df_paris_district.loc[0, 'Arrondissement'] # neighborhood name

print('Latitude and longitude values of Arrondissement {} are {}, {}.'.format(arr_name, 
                                                               arr_latitude, 
                                                               arr_longitude))

Latitude and longitude values of Arrondissement 1 are 48.862497025730065, 2.336686138746136.


#### Now, let's get the top 100 Bubble tea shops that are in arrondissement 1 within a radius of 800 meters

In [109]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 800 # define radius

categoryId_bubbleTea = '52e81612bcbc57f1066b7a0c'

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={} \
&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    arr_latitude, 
    arr_longitude, 
    radius, 
    LIMIT,
    categoryId_bubbleTea
)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=EMC0WNWPGRILVV5GXBNFYYWLH0OJIRSC5BJRVBBMPOHWADEU&client_secret=I31543KR20OXAXEUJTI5WEHN0F3RTH5LXMXL1IHYSXVETIHO&v=20180605&ll=48.862497025730065,2.336686138746136&radius=800&limit=100 &categoryId=52e81612bcbc57f1066b7a0c'

In [110]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0b1d36461da410d6f923ad'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Palais-Royal',
  'headerFullLocation': 'Palais-Royal, Paris',
  'headerLocationGranularity': 'neighborhood',
  'query': 'bubble tea',
  'totalResults': 14,
  'suggestedBounds': {'ne': {'lat': 48.86969703293007,
    'lng': 2.3476101541074796},
   'sw': {'lat': 48.85529701853006, 'lng': 2.3257621233847927}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5001951de4b04575fdb89702',
       'name': 'ZenZoo – Bar à Thé',
       'location': {'address': '2 rue Chabanais',
        'lat': 48.867020860798846,
        'lng': 2.336737853474915,
        'labeledLatLngs': [{'label': '

In [111]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [112]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.categories.value_counts()

Bubble Tea Shop         9
Taiwanese Restaurant    2
Chinese Restaurant      1
Dessert Shop            1
Tea Room                1
Name: categories, dtype: int64

In [113]:
nearby_venues.loc[nearby_venues['categories'] == 'Bubble Tea Shop']

,name,categories,lat,lng
0,ZenZoo – Bar à Thé,Bubble Tea Shop,48.867021,2.336738
1,Chatime,Bubble Tea Shop,48.865320,2.336665
3,TeaYammi,Bubble Tea Shop,48.866760,2.335194
4,Le 37m²,Bubble Tea Shop,48.868636,2.336722
5,Momen’Tea,Bubble Tea Shop,48.864941,2.334792
6,The Alley,Bubble Tea Shop,48.867163,2.334782
8,Yi Yun,Bubble Tea Shop,48.866037,2.335741
9,Zen Zoo,Bubble Tea Shop,48.864630,2.340845
12,Moment’Tea Forum des Halles,Bubble Tea Shop,48.861884,2.347017


In [114]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

14 venues were returned by Foursquare.


In [115]:
nearby_venues

,name,categories,lat,lng
0,ZenZoo – Bar à Thé,Bubble Tea Shop,48.867021,2.336738
1,Chatime,Bubble Tea Shop,48.865320,2.336665
2,ZenZoo,Taiwanese Restaurant,48.867771,2.337020
3,TeaYammi,Bubble Tea Shop,48.866760,2.335194
4,Le 37m²,Bubble Tea Shop,48.868636,2.336722
5,Momen’Tea,Bubble Tea Shop,48.864941,2.334792
6,The Alley,Bubble Tea Shop,48.867163,2.334782
7,Momo,Taiwanese Restaurant,48.869369,2.334601
8,Yi Yun,Bubble Tea Shop,48.866037,2.335741
9,Zen Zoo,Bubble Tea Shop,48.864630,2.340845


#### Let's create a function to repeat the same process to all the districts in Paris

In [122]:
def dist_radius(number): # depends on the diameter of the district => sqrt(superficy/np.pi) = radius
    radius = 0
    if number == 1:
        radius = 760
    elif number == 2:
        radius = 560
    elif number == 3:
        radius = 610
    elif number == 4:
        radius = 710
    elif number == 5:
        radius = 900
    elif number == 6:
        radius = 827
    elif number == 7:
        radius = 1141
    elif number == 8:
        radius = 1110
    elif number == 9:
        radius = 833
    elif number == 10:
        radius = 959
    elif number == 11:
        radius = 1080
    elif number == 12:
        radius = 1420
    elif number == 13:
        radius = 1510
    elif number == 14:
        radius = 1340
    elif number == 15:
        radius = 1643
    elif number == 16:
        radius = 1580
    elif number == 17:
        radius = 1343
    elif number == 18:
        radius = 1380
    elif number == 19:
        radius = 1470
    elif number == 20:
        radius = 1380
    return(radius)

In [138]:
def getNearbyVenues(names, latitudes, longitudes, categoryId_bubble):
    
    venues_list=[]
    count = 1
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            dist_radius(count), 
            LIMIT,
            categoryId_bubble
        )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        count = count + 1

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Arrondissement', 
                  'Arrondissement Latitude', 
                  'Arrondissement Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [184]:
# type your answer here

paris_venues = getNearbyVenues(names=df_paris_district['Arrondissement'],
                                   latitudes=df_paris_district['Latitude'],
                                   longitudes=df_paris_district['Longitude'],
                                   categoryId_bubble = '52e81612bcbc57f1066b7a0c'
                                  )



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [178]:
print(paris_venues.shape)
paris_venues

(85, 7)


,Arrondissement,Arrondissement Latitude,Arrondissement Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,1,48.862497,2.336686,ZenZoo – Bar à Thé,48.867021,2.336738,Bubble Tea Shop
1,1,48.862497,2.336686,Chatime,48.865320,2.336665,Bubble Tea Shop
2,1,48.862497,2.336686,ZenZoo,48.867771,2.337020,Taiwanese Restaurant
3,1,48.862497,2.336686,TeaYammi,48.866760,2.335194,Bubble Tea Shop
4,1,48.862497,2.336686,Le 37m²,48.868636,2.336722,Bubble Tea Shop
5,1,48.862497,2.336686,Momen’Tea,48.864941,2.334792,Bubble Tea Shop
6,1,48.862497,2.336686,The Alley,48.867163,2.334782,Bubble Tea Shop
7,1,48.862497,2.336686,Yi Yun,48.866037,2.335741,Bubble Tea Shop
8,1,48.862497,2.336686,Zen Zoo,48.864630,2.340845,Bubble Tea Shop
9,1,48.862497,2.336686,Crazy Mango,48.867123,2.334938,Dessert Shop


In [135]:
print('{} venues were returned by Foursquare.'.format(paris_venues.shape[0]))

85 venues were returned by Foursquare.


In [190]:
# There are some venues with an other category that is not 'Bubble tea shop'
paris_venues = paris_venues[paris_venues['Venue Category'] == 'Bubble Tea Shop']
paris_venues.head(15)

,Arrondissement,Arrondissement Latitude,Arrondissement Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,1,48.862497,2.336686,ZenZoo – Bar à Thé,48.867021,2.336738,Bubble Tea Shop
1,1,48.862497,2.336686,Chatime,48.865320,2.336665,Bubble Tea Shop
3,1,48.862497,2.336686,TeaYammi,48.866760,2.335194,Bubble Tea Shop
4,1,48.862497,2.336686,Le 37m²,48.868636,2.336722,Bubble Tea Shop
5,1,48.862497,2.336686,Momen’Tea,48.864941,2.334792,Bubble Tea Shop
6,1,48.862497,2.336686,The Alley,48.867163,2.334782,Bubble Tea Shop
7,1,48.862497,2.336686,Yi Yun,48.866037,2.335741,Bubble Tea Shop
8,1,48.862497,2.336686,Zen Zoo,48.864630,2.340845,Bubble Tea Shop
11,1,48.862497,2.336686,Moment’Tea Forum des Halles,48.861884,2.347017,Bubble Tea Shop
12,2,48.866451,2.341356,ZenZoo – Bar à Thé,48.867021,2.336738,Bubble Tea Shop


In [151]:
print('{} venues were returned by Foursquare.'.format(paris_venues.shape[0]))

56 venues were returned by Foursquare.


#### Some rows appeared several times in different districts because of the approximation of the radius

In [189]:
print(paris_venues.Venue.unique())
print(paris_venues.Venue.unique().shape) # 36 uniques venues !

['ZenZoo – Bar à Thé' 'Chatime' 'TeaYammi' 'Le 37m²' 'Momen’Tea'
 'The Alley' 'Yi Yun' 'Zen Zoo' 'Moment’Tea Forum des Halles'
 'Laïzé 來座 (Laïzé)' 'Miruku Ti' 'Xing Fu Tang Bubble Tea'
 'MA+CH （MATCH Tea）' 'théobul' 'Meng Salon de Thé' 'Teawan'
 'Chamie Bubble Tea' "Bubble N' Tea" 'Tchat Huant' 'NEKOTEA'
 'Maracuja Odeon' 'Tea And Waffle' "O'naicha 香閣" 'Bubble Shop' 'Thé Point'
 'Chatime Voltaire' 'Sin Heaven' 'Bubble Wrap' 'Boba Tea Shop' 'TeaWan'
 'Bubble T. Paris' 'Bubbleology' 'Bon goûthé' 'Coco Lafayette'
 'Bubble Vie' 'M63 Paris Salon De Thé']
(36,)


In [198]:
paris_venues_clean = paris_venues.drop_duplicates(subset=['Venue', 'Venue Category'], keep='first')
paris_venues_clean.reset_index(drop=True)
paris_venues_clean.sort_values(by=['Venue'])

,Arrondissement,Arrondissement Latitude,Arrondissement Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
69,13,48.831586,2.355638,Boba Tea Shop,48.824254,2.366259,Bubble Tea Shop
77,15,48.841821,2.299106,Bon goûthé,48.845575,2.288094,Bubble Tea Shop
36,5,48.845428,2.344691,Bubble N' Tea,48.839277,2.339911,Bubble Tea Shop
42,8,48.877823,2.315785,Bubble Shop,48.870227,2.308218,Bubble Tea Shop
73,14,48.832754,2.325071,Bubble T. Paris,48.841039,2.329861,Bubble Tea Shop
81,18,48.892276,2.344038,Bubble Vie,48.884746,2.340215,Bubble Tea Shop
62,12,48.840474,2.388242,Bubble Wrap,48.831245,2.380666,Bubble Tea Shop
74,14,48.832754,2.325071,Bubbleology,48.837653,2.314382,Bubble Tea Shop
35,5,48.845428,2.344691,Chamie Bubble Tea,48.851519,2.346061,Bubble Tea Shop
1,1,48.862497,2.336686,Chatime,48.865320,2.336665,Bubble Tea Shop


In [202]:
print(paris_venues_clean.shape)
# Now we have the good dataset

(36, 7)


#### Let's check how many venues were returned for each district

In [200]:
paris_venues_clean.groupby('Arrondissement').count()['Venue']

Arrondissement
1     9
3     4
4     2
5     3
6     4
8     2
11    3
12    1
13    2
14    2
15    1
17    1
18    2
Name: Venue, dtype: int64

#### Print map of venues of bubble tea shop

In [379]:
# create map
map_venues_paris = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to the map
for lat, lon, poi in zip(paris_venues_clean['Venue Latitude'], paris_venues_clean['Venue Longitude'], paris_venues_clean['Arrondissement']):
    label = '{}, Arrondissement {}'.format(address, poi)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_venues_paris)
       
map_venues_paris 

## Classification

In [204]:
# We would like to predict if a restaurant can be open in a district under the number of bubble tea shop per district

In [336]:
df_paris_count = paris_venues_clean.groupby('Arrondissement').count()['Venue']
df_paris_count = df_paris_count.to_frame()
df_paris_count.reset_index(drop=False, inplace=True)
df_paris_count

,Arrondissement,Venue
0,1,9
1,3,4
2,4,2
3,5,3
4,6,4
5,8,2
6,11,3
7,12,1
8,13,2
9,14,2


In [337]:
numberVenue = np.zeros(20)
for arr, element in zip(df_paris_count['Arrondissement'], df_paris_count['Venue']):
    numberVenue[arr] = element

numberVenue

array([0., 9., 0., 4., 2., 3., 4., 0., 2., 0., 0., 3., 1., 2., 2., 1., 0.,
       1., 2., 0.])

In [338]:
tab = range(1,21)
venue = np.zeros(20)
df_paris_count_district = pd.DataFrame({'Arrondissement':tab, 'Venue':numberVenue})
df_paris_count_district

,Arrondissement,Venue
0,1,0.0
1,2,9.0
2,3,0.0
3,4,4.0
4,5,2.0
5,6,3.0
6,7,4.0
7,8,0.0
8,9,2.0
9,10,0.0


If a district have more than 3 Bubble Tea Shop, we will give a rate of 1 otherwise 0.

Rate == Y_train

In [339]:
df_paris_count_district.loc[df_paris_count_district['Venue'] > 3, 'Rate'] = 1
df_paris_count_district.loc[df_paris_count_district['Venue'] <= 3, 'Rate'] = 0
df_paris_count_district

,Arrondissement,Venue,Rate
0,1,0.0,0.0
1,2,9.0,1.0
2,3,0.0,0.0
3,4,4.0,1.0
4,5,2.0,0.0
5,6,3.0,0.0
6,7,4.0,1.0
7,8,0.0,0.0
8,9,2.0,0.0
9,10,0.0,0.0


In [343]:
# We Merged with the location of each district
df_paris_count_district = df_paris_count_district.merge(df_paris_district,on=['Arrondissement'])
df_paris_count_district

,Arrondissement,Venue,Rate,Latitude,Longitude
0,1,0.0,0.0,48.862497,2.336686
1,2,9.0,1.0,48.866451,2.341356
2,3,0.0,0.0,48.863642,2.361263
3,4,4.0,1.0,48.854662,2.362913
4,5,2.0,0.0,48.845428,2.344691
5,6,3.0,0.0,48.850414,2.331738
6,7,4.0,1.0,48.857859,2.314381
7,8,0.0,0.0,48.877823,2.315785
8,9,2.0,0.0,48.872839,2.339630
9,10,0.0,0.0,48.871760,2.356860


#### Feature selection

In [347]:
X = df_paris_count_district.loc[:,['Arrondissement', 'Venue', 'Latitude', 'Longitude']]
X

,Arrondissement,Venue,Latitude,Longitude
0,1,0.0,48.862497,2.336686
1,2,9.0,48.866451,2.341356
2,3,0.0,48.863642,2.361263
3,4,4.0,48.854662,2.362913
4,5,2.0,48.845428,2.344691
5,6,3.0,48.850414,2.331738
6,7,4.0,48.857859,2.314381
7,8,0.0,48.877823,2.315785
8,9,2.0,48.872839,2.339630
9,10,0.0,48.871760,2.356860


In [354]:
Y_train = df_paris_count_district['Rate']
Y_train

0     0.0
1     1.0
2     0.0
3     1.0
4     0.0
5     0.0
6     1.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
Name: Rate, dtype: float64

#### Normalize the data

In [355]:
from sklearn import preprocessing
X_train= preprocessing.StandardScaler().fit(X).transform(X)
X_train

/Users/J-Luc/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/J-Luc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


array([[-1.64750894, -0.8523247 ,  0.09059069, -0.23267278],
       [-1.47408695,  3.40929881,  0.32658265, -0.07713582],
       [-1.30066495, -0.8523247 ,  0.15895735,  0.58597634],
       [-1.12724296,  1.04173019, -0.37706443,  0.64094532],
       [-0.95382097,  0.09470274, -0.92820907,  0.03397478],
       [-0.78039897,  0.56821647, -0.63062601, -0.39750154],
       [-0.60697698,  1.04173019, -0.1862622 , -0.97564742],
       [-0.43355498, -0.8523247 ,  1.00538672, -0.92888694],
       [-0.26013299,  0.09470274,  0.70790078, -0.1346176 ],
       [-0.086711  , -0.8523247 ,  0.64350872,  0.43932911],
       [ 0.086711  , -0.8523247 , -0.14489326,  1.18060621],
       [ 0.26013299,  0.56821647, -1.22392513,  1.48463313],
       [ 0.43355498, -0.37881098, -1.75446454,  0.39862169],
       [ 0.60697698,  0.09470274, -1.68472518, -0.61956638],
       [ 0.78039897,  0.09470274, -1.14355616, -1.48446226],
       [ 0.95382097, -0.37881098,  0.17572952, -2.22491293],
       [ 1.12724296, -0.

### KNN

In [356]:
from sklearn.neighbors import KNeighborsClassifier
k = 3
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,Y_train)
neigh

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [358]:
X_test = [[-0.60697698, 0.47470274, 0.09059069, -0.23267278]]
yhat = neigh.predict(X_test)
print(yhat)

[0.]


In [359]:
# With these randoms values of the location, we can deduce that it will be less than 3 bubble tea shops at that location.

In [376]:
# Geographical location of the invalides in the 7th district
X_test_district_7th = pd.DataFrame({'Arrondissement':[7.0],'Venue':[0.0], 'Latitude': [48.8584991851266], 'Longitude': [2.3131056236284486]})
X_test2 = preprocessing.StandardScaler().fit(X_test_district_7th).transform(X_test_district_7th)
X_test2

array([[0., 0., 0., 0.]])

In [377]:
yhat2 = neigh.predict(X_test2)
print(yhat)

[0.]


In [378]:
# It is a good idea to open a bubble tea shop around the invalides